# Small preprocessing of PAS data

In [1]:
import pandas as pd
import os

In [2]:
# loading the data

data_local = pd.read_csv(".\data\PAS_Q3_23_24_Borough.csv")
data_MPS = pd.read_csv(".\data\PAS_Q3_23_24_MPS.csv")

C:\Users\Marceli Morawski\AppData\Local\Temp\ipykernel_31368\2478593547.py:3: DtypeWarning: Columns (0,1,2,3,8) have mixed types. Specify dtype option on import or set low_memory=False.
  data_local = pd.read_csv("PAS_Q3_23_24_Borough.csv")


In [3]:
# modifing data type in colums and deleting for MPS
data_MPS = data_MPS.drop(columns=["Survey", "Borough"])
data_MPS["Date"] = pd.to_datetime(data_MPS["Date"], dayfirst=True)
data_MPS

,Date,Measure,Proportion
0,2014-12-31,"""Good Job"" local",0.67
1,2014-12-31,Contact ward officer,0.41
2,2014-12-31,Informed local,0.49
3,2014-12-31,Listen to concerns,0.74
4,2014-12-31,Relied on to be there,0.77
...,...,...,...
286,2023-12-31,Listen to concerns,0.57
287,2023-12-31,Relied on to be there,0.59
288,2023-12-31,Treat everyone fairly,0.62
289,2023-12-31,Trust MPS,0.69


In [4]:
# modifing data type in colums and deleting for MPS
data_local = data_local[~data_local["Date"].isna()].drop(columns=data_local.columns[[1] + list(range(6, 10))])
data_local["Date"] = pd.to_datetime(data_local["Date"], dayfirst=True)
data_local

,Date,Borough,Measure,Proportion,MPS
0,2014-12-31,Barking and Dagenham,"""Good Job"" local",0.56,0.67
1,2014-12-31,Barnet,"""Good Job"" local",0.67,0.67
2,2014-12-31,Bexley,"""Good Job"" local",0.72,0.67
3,2014-12-31,Brent,"""Good Job"" local",0.66,0.67
4,2014-12-31,Bromley,"""Good Job"" local",0.71,0.67
...,...,...,...,...,...
9307,2023-12-31,Sutton,Understand issues,0.66,0.57
9308,2023-12-31,Tower Hamlets,Understand issues,0.55,0.57
9309,2023-12-31,Waltham Forest,Understand issues,0.39,0.57
9310,2023-12-31,Wandsworth,Understand issues,0.56,0.57


In [5]:
# lazy check for data types
print(data_MPS.dtypes)
print("###############################")
print(data_local.dtypes)
# data types seem to be correct

Date          datetime64[ns]
Measure               object
Proportion           float64
dtype: object
###############################
Date          datetime64[ns]
Borough               object
Measure               object
Proportion           float64
MPS                  float64
dtype: object


In [6]:
# checking for NANs
print(data_MPS.isnull().any())
    
print(data_local.isnull().any())

Date          False
Measure       False
Proportion    False
dtype: bool
Date          False
Borough       False
Measure       False
Proportion    False
MPS           False
dtype: bool


In [7]:
# calculate IQR for column Proportion # Just checking if there are any strange values like 1% or 1000%
Q1 = data_MPS["Proportion"].quantile(0.25)
Q3 = data_MPS['Proportion'].quantile(0.75)
IQR = Q3 - Q1

# identify outliers
threshold = 1.4
outliers = data_MPS[(data_MPS['Proportion'] < Q1 - threshold * IQR) | (data_MPS['Proportion'] > Q3 + threshold * IQR)]
print(outliers)

Empty DataFrame
Columns: [Date, Measure, Proportion]
Index: []


In [8]:
# calculate IQR for column Proportion # Just checking if there are any strange values like 1% or 1000%
Q1 = data_local["Proportion"].quantile(0.25)
Q3 = data_local['Proportion'].quantile(0.75)
IQR = Q3 - Q1

# identify outliers
threshold = 1.4
outliers = data_local[(data_local['Proportion'] < Q1 - threshold * IQR) | (data_local['Proportion'] > Q3 + threshold * IQR)]
print(outliers["Measure"].unique())
print(outliers["Borough"].unique()) # Intresting small results with Contact ward ofiicer

# Next thing we should check yearly results

['Contact ward officer']
['City of Westminster' 'Greenwich' 'Kensington and Chelsea' 'Merton'
 'Newham' 'Richmond upon Thames' 'Southwark' 'Tower Hamlets' 'Wandsworth'
 'Hammersmith and Fulham' 'Harrow' 'Brent' 'Hounslow'
 'Barking and Dagenham' 'Lambeth' 'Croydon' 'Ealing' 'Hillingdon'
 'Lewisham']


In [9]:
# listing all measures and checking if the data  has the same attr.:
data_local["Measure"].unique() == data_MPS["Measure"].unique()
data_local["Measure"].unique()

array(['"Good Job" local', 'Contact ward officer', 'Informed local',
       'Listen to concerns', 'Relied on to be there',
       'Treat everyone fairly', 'Understand issues', 'Trust MPS'],
      dtype=object)

In [18]:
data_local[(data_local["Borough"] == "Lambeth") & (data_local["Measure"] == "Trust MPS")]

,Date,Borough,Measure,Proportion,MPS
6740,2016-03-31,Lambeth,Trust MPS,0.85,0.86
6772,2016-06-30,Lambeth,Trust MPS,0.89,0.87
6804,2016-09-30,Lambeth,Trust MPS,0.88,0.87
6836,2016-12-31,Lambeth,Trust MPS,0.88,0.88
6868,2017-03-31,Lambeth,Trust MPS,0.89,0.88
6900,2017-06-30,Lambeth,Trust MPS,0.88,0.88
6932,2017-09-30,Lambeth,Trust MPS,0.88,0.87
6964,2017-12-31,Lambeth,Trust MPS,0.90,0.86
6996,2018-03-31,Lambeth,Trust MPS,0.87,0.86
7028,2018-06-30,Lambeth,Trust MPS,0.86,0.85


In [97]:
#### download preprocessed data

data_MPS.to_csv(path_or_buf = "data_PAS_MPS.csv")
data_local.to_csv(path_or_buf = "data_PAS_Borough.csv")